In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
from workalendar.asia import SouthKorea
import pendulum

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [4]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

# train['중식참여율'] = train['중식계'] / train['식사가능자수']

<ipython-input-4-b11c832c5082>:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  train['주'] = pd.DatetimeIndex(train['일자']).week
<ipython-input-4-b11c832c5082>:4: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  test['주'] = pd.DatetimeIndex(test['일자']).week


In [5]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [6]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

In [7]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3
    if tomorrow in holidays:
        return 2
    elif yesterday in holidays:
        return 1
    else : 
        return 0

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

## 공휴일 변수 생성

In [8]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '월(석식)', '월(중식)',
       '요일(석식)', '요일(중식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차', '인원변화'],
      dtype='object')

In [9]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_train = train[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
lunch_test = test[['공휴일전후', '몇주차', '인원변화', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

In [10]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
      dtype='object')

In [11]:
print(lunch_train.shape)
print(lunch_test.shape)

(1187, 16)
(50, 16)


In [12]:
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 16)
(50, 16)


In [13]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: []
[]


In [14]:
y_lunch = train[['중식계']]
y_dinner = train[['석식계']]

#### 분포 확인 및 분포 조정

# 중식 예측모델

In [67]:
import optuna
from xgboost import XGBRegressor
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold

In [68]:
# from automl_alex import LightGBMRegressor, CatBoostRegressor, AutoMLRegressor
from sklearn.metrics import mean_absolute_error as MAE
import sklearn

In [93]:
y_lunch = np.array(y_lunch).reshape(-1,1)
def objective_lunch(trial: Trial) -> float:
    params_xgb = {
        "random_state": 42,
        "verbosity": 1,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators": 10000,
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'min_child_weight' : trial.suggest_float("min_child_weight", 0.1, 3.0),
        'metrics' : MAE
    }
    n_fold = 8
    kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)
    log_scores = []

    for train_index, test_index in kf.split(lunch_train):
        X_train, X_valid = lunch_train.iloc[train_index], lunch_train.iloc[test_index]
        y_train, y_valid = y_lunch[train_index], y_lunch[test_index]
        
        model = XGBRegressor(**params_xgb)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=False,
        )
    
        xgb_pred = model.predict(X_valid)
        log_score = MAE(y_valid, xgb_pred)
        log_scores.append(log_score)
        
     
    
    return np.mean(log_scores)

In [94]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective_lunch, n_trials=100)
lunch_param = study.best_trial.params

print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-20 22:14:47,725] A new study created in memory with name: lgbm_parameter_opt
[I 2021-07-20 22:14:53,228] Trial 0 finished with value: 68.74942965484155 and parameters: {'learning_rate': 0.03807947176588889, 'reg_alpha': 2.8521922049233386e-05, 'reg_lambda': 0.06587945744308704, 'max_depth': 12, 'colsample_bytree': 0.4936111842654619, 'subsample': 0.40919616423534183, 'max_bin': 217, 'min_child_weight': 2.611910822747312}. Best is trial 0 with value: 68.74942965484155.
[I 2021-07-20 22:15:02,782] Trial 1 finished with value: 69.73767255781004 and parameters: {'learning_rate': 0.06051038616257767, 'reg_alpha': 2.1245096608103405e-05, 'reg_lambda': 0.0018526142807772773, 'max_depth': 20, 'colsample_bytree': 0.899465584480253, 'subsample': 0.44863737747479326, 'max_bin': 254, 'min_child_weight': 0.631873078574958}. Best is trial 0 with value: 68.74942965484155.
[I 2021-07-20 22:15:07,290] Trial 2 finished with value: 65.97919418736981 and parameters: {'learning_rate': 0.03111998

[I 2021-07-20 22:19:57,531] Trial 20 finished with value: 69.26719002331879 and parameters: {'learning_rate': 0.04996411792689219, 'reg_alpha': 3.4801917314941598e-06, 'reg_lambda': 0.051542220607625325, 'max_depth': 9, 'colsample_bytree': 0.9956616110467869, 'subsample': 0.7786131264093644, 'max_bin': 493, 'min_child_weight': 2.187006787496504}. Best is trial 17 with value: 65.1760790238639.
[I 2021-07-20 22:20:00,412] Trial 21 finished with value: 64.98255074696685 and parameters: {'learning_rate': 0.0843480751195553, 'reg_alpha': 2.552999329978968e-07, 'reg_lambda': 0.06034895610579548, 'max_depth': 3, 'colsample_bytree': 0.9139596024275602, 'subsample': 0.7004119961062609, 'max_bin': 231, 'min_child_weight': 2.892219344257274}. Best is trial 21 with value: 64.98255074696685.
[I 2021-07-20 22:20:03,187] Trial 22 finished with value: 65.70602134531258 and parameters: {'learning_rate': 0.09731874712529351, 'reg_alpha': 7.468949518842916e-07, 'reg_lambda': 0.05981518410570621, 'max_dep

[I 2021-07-20 22:21:28,972] Trial 41 finished with value: 66.05514420395687 and parameters: {'learning_rate': 0.061310967829780925, 'reg_alpha': 4.934782110627822e-06, 'reg_lambda': 0.05675173916869637, 'max_depth': 4, 'colsample_bytree': 0.9634157742212522, 'subsample': 0.5117237607281847, 'max_bin': 324, 'min_child_weight': 2.552967185419378}. Best is trial 26 with value: 64.86522512126267.
[I 2021-07-20 22:21:31,787] Trial 42 finished with value: 65.6695940198298 and parameters: {'learning_rate': 0.07653245962381335, 'reg_alpha': 3.7208041194412824e-06, 'reg_lambda': 0.06300198770769205, 'max_depth': 3, 'colsample_bytree': 0.9890099443022972, 'subsample': 0.6085547221109969, 'max_bin': 334, 'min_child_weight': 2.8064893548077277}. Best is trial 26 with value: 64.86522512126267.
[I 2021-07-20 22:21:35,115] Trial 43 finished with value: 65.89037702703311 and parameters: {'learning_rate': 0.06915784235784568, 'reg_alpha': 1.864142301055956e-06, 'reg_lambda': 0.06880719008559659, 'max_d

[I 2021-07-20 22:24:50,869] Trial 62 finished with value: 64.94341748737713 and parameters: {'learning_rate': 0.007944409023485985, 'reg_alpha': 2.871571788394278e-06, 'reg_lambda': 0.011759125869213195, 'max_depth': 4, 'colsample_bytree': 0.6809274244959025, 'subsample': 0.7796424084403297, 'max_bin': 245, 'min_child_weight': 1.4922036452864156}. Best is trial 53 with value: 64.84087550901748.
[I 2021-07-20 22:25:07,999] Trial 63 finished with value: 65.1880562676939 and parameters: {'learning_rate': 0.006745273037043657, 'reg_alpha': 2.910129613070751e-06, 'reg_lambda': 0.01916838152134782, 'max_depth': 4, 'colsample_bytree': 0.6880654679780288, 'subsample': 0.8203719399331474, 'max_bin': 227, 'min_child_weight': 1.2411375646866176}. Best is trial 53 with value: 64.84087550901748.
[I 2021-07-20 22:25:22,937] Trial 64 finished with value: 65.68600178923901 and parameters: {'learning_rate': 0.007199805092705569, 'reg_alpha': 3.1813063253495585e-06, 'reg_lambda': 0.005307670042142526, '

[I 2021-07-20 22:29:36,599] Trial 83 finished with value: 64.70361708228134 and parameters: {'learning_rate': 0.02016092881099785, 'reg_alpha': 7.391123373447245e-06, 'reg_lambda': 0.026044164587402892, 'max_depth': 3, 'colsample_bytree': 0.5930542578096945, 'subsample': 0.8373644224791007, 'max_bin': 259, 'min_child_weight': 2.4283241820252464}. Best is trial 83 with value: 64.70361708228134.
[I 2021-07-20 22:29:47,203] Trial 84 finished with value: 65.53769547386176 and parameters: {'learning_rate': 0.021443864100609625, 'reg_alpha': 1.0860231532132474e-05, 'reg_lambda': 0.0250850613492829, 'max_depth': 2, 'colsample_bytree': 0.5787228944532549, 'subsample': 0.8376655975424742, 'max_bin': 287, 'min_child_weight': 2.446398984187769}. Best is trial 83 with value: 64.70361708228134.
[I 2021-07-20 22:29:58,397] Trial 85 finished with value: 65.75758186511933 and parameters: {'learning_rate': 0.012241871503559808, 'reg_alpha': 7.078339503115714e-06, 'reg_lambda': 0.026516965662996324, 'ma

Best Score: 64.47039592320141
Best trial: {'learning_rate': 0.024173663455776283, 'reg_alpha': 7.420901336616963e-06, 'reg_lambda': 0.020012251064973507, 'max_depth': 3, 'colsample_bytree': 0.633687844825795, 'subsample': 0.7519217106315786, 'max_bin': 275, 'min_child_weight': 2.3315692557276133}


In [95]:
X_train, X_valid, y_train, y_valid = train_test_split(lunch_train, y_lunch, test_size=0.2)

lunch_model = XGBRegressor(**lunch_param)
lunch_model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=False,
        )

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.633687844825795, gamma=0,
       gpu_id=-1, importance_type='gain', interaction_constraints='',
       learning_rate=0.024173663455776283, max_bin=275, max_delta_step=0,
       max_depth=3, min_child_weight=2.3315692557276133, missing=nan,
       monotone_constraints='()', n_estimators=100, n_jobs=8,
       num_parallel_tree=1, objective='reg:squarederror', random_state=0,
       reg_alpha=7.420901336616963e-06, reg_lambda=0.020012251064973507,
       scale_pos_weight=1, subsample=0.7519217106315786,
       tree_method='exact', validate_parameters=1, verbosity=None)

In [96]:
lunch_predict = lunch_model.predict(lunch_test)
lunch_predict

array([ 897.7824 ,  869.88824,  613.5938 , 1108.5725 ,  951.48663,
        914.88666,  903.12494,  631.5042 , 1128.2795 ,  936.99384,
        730.63434, 1118.4377 ,  993.33136,  942.6508 ,  851.7378 ,
        635.1935 , 1110.2025 ,  961.43317,  871.0109 ,  833.01434,
        583.0444 ,  981.7322 ,  917.84204,  862.03265,  628.7478 ,
       1115.0837 ,  986.8131 ,  917.84204,  882.0774 ,  657.9067 ,
       1118.0635 ,  931.4208 ,  900.0161 ,  854.7093 ,  608.9942 ,
       1097.6671 ,  925.17596,  852.8931 ,  801.5344 ,  587.48596,
       1086.4932 ,  926.10474,  890.12445,  792.79956,  584.48444,
       1084.7614 ,  907.40436,  868.2821 ,  808.8119 ,  582.27637],
      dtype=float32)

In [44]:
def objective_dinner(trial: Trial) -> float:
    params_xgb = {
        "random_state": 42,
        "verbosity": 1,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators": 10000,
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'min_child_weight' : trial.suggest_float("min_child_weight", 0.1, 3.0),
        'metrics' : MAE
    }
    
    X_train, X_valid, y_train, y_valid = train_test_split(dinner_train, y_dinner, test_size=0.2)

    model = XGBRegressor(**params_xgb)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=100,
        verbose=False,
    
    )

    lgb_pred = model.predict(X_valid)
    log_score = MAE(y_valid, lgb_pred)
    
    return log_score

In [45]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective_dinner, n_trials=100)
dinner_param = study.best_trial.params

print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-20 21:07:37,187] A new study created in memory with name: lgbm_parameter_opt
[I 2021-07-20 21:07:37,888] Trial 0 finished with value: 54.251481513015364 and parameters: {'learning_rate': 0.03807947176588889, 'reg_alpha': 2.8521922049233386e-05, 'reg_lambda': 0.06587945744308704, 'max_depth': 12, 'colsample_bytree': 0.4936111842654619, 'subsample': 0.40919616423534183, 'max_bin': 217, 'min_child_weight': 2.611910822747312}. Best is trial 0 with value: 54.251481513015364.
[I 2021-07-20 21:07:38,805] Trial 1 finished with value: 62.657777032932316 and parameters: {'learning_rate': 0.06051038616257767, 'reg_alpha': 2.1245096608103405e-05, 'reg_lambda': 0.0018526142807772773, 'max_depth': 20, 'colsample_bytree': 0.899465584480253, 'subsample': 0.44863737747479326, 'max_bin': 254, 'min_child_weight': 0.631873078574958}. Best is trial 0 with value: 54.251481513015364.
[I 2021-07-20 21:07:39,292] Trial 2 finished with value: 54.07336882583233 and parameters: {'learning_rate': 0.0311

[I 2021-07-20 21:08:31,829] Trial 20 finished with value: 70.1085804945708 and parameters: {'learning_rate': 0.09261284238653544, 'reg_alpha': 9.51201346804592e-06, 'reg_lambda': 0.036206363887310855, 'max_depth': 20, 'colsample_bytree': 0.7984738147520278, 'subsample': 0.9965184544294958, 'max_bin': 458, 'min_child_weight': 0.13091605011783924}. Best is trial 5 with value: 51.145048638351824.
[I 2021-07-20 21:08:34,936] Trial 21 finished with value: 55.574447856229895 and parameters: {'learning_rate': 0.009043520647847674, 'reg_alpha': 2.7164581657755197e-05, 'reg_lambda': 0.07517849498237927, 'max_depth': 17, 'colsample_bytree': 0.4811383250833349, 'subsample': 0.7256030936291626, 'max_bin': 386, 'min_child_weight': 1.672815980728206}. Best is trial 5 with value: 51.145048638351824.
[I 2021-07-20 21:08:38,122] Trial 22 finished with value: 55.39253795848173 and parameters: {'learning_rate': 0.008867114683573808, 'reg_alpha': 2.6117597384123255e-05, 'reg_lambda': 0.07202806719161609, 

[I 2021-07-20 21:09:14,605] Trial 40 finished with value: 53.40631634848459 and parameters: {'learning_rate': 0.022387650476561355, 'reg_alpha': 1.6212073403756604e-05, 'reg_lambda': 0.08445824588453703, 'max_depth': 9, 'colsample_bytree': 0.9128064092914325, 'subsample': 0.9641094748034411, 'max_bin': 355, 'min_child_weight': 1.211596144928233}. Best is trial 26 with value: 49.85171074426475.
[I 2021-07-20 21:09:18,532] Trial 41 finished with value: 56.42806634983095 and parameters: {'learning_rate': 0.004970714030221816, 'reg_alpha': 2.802014057800947e-05, 'reg_lambda': 0.08136548756465795, 'max_depth': 16, 'colsample_bytree': 0.7301555379847198, 'subsample': 0.444971358274279, 'max_bin': 305, 'min_child_weight': 1.5109133510165134}. Best is trial 26 with value: 49.85171074426475.
[I 2021-07-20 21:09:26,725] Trial 42 finished with value: 60.34386115514931 and parameters: {'learning_rate': 0.012177643998402862, 'reg_alpha': 2.5110159483891742e-05, 'reg_lambda': 0.08927540861952268, 'm

[I 2021-07-20 21:10:25,762] Trial 60 finished with value: 59.24178697281525 and parameters: {'learning_rate': 0.09315292205348734, 'reg_alpha': 9.711986137225232e-06, 'reg_lambda': 0.05056192138763832, 'max_depth': 9, 'colsample_bytree': 0.8672071006511379, 'subsample': 0.6197681510144908, 'max_bin': 499, 'min_child_weight': 1.6801875538524407}. Best is trial 53 with value: 47.77141155114695.
[I 2021-07-20 21:10:26,332] Trial 61 finished with value: 56.12855645287939 and parameters: {'learning_rate': 0.08406561738114915, 'reg_alpha': 2.406434465455747e-05, 'reg_lambda': 0.07583817424726556, 'max_depth': 11, 'colsample_bytree': 0.8927722008044543, 'subsample': 0.725082475771599, 'max_bin': 478, 'min_child_weight': 1.2769893174914964}. Best is trial 53 with value: 47.77141155114695.
[I 2021-07-20 21:10:27,001] Trial 62 finished with value: 54.18274007524763 and parameters: {'learning_rate': 0.0960241336602289, 'reg_alpha': 2.7162964632684825e-05, 'reg_lambda': 0.08166001132626696, 'max_d

[I 2021-07-20 21:10:36,924] Trial 81 finished with value: 57.51255909735415 and parameters: {'learning_rate': 0.046859790704736136, 'reg_alpha': 8.093204225986034e-06, 'reg_lambda': 0.0737655891401602, 'max_depth': 7, 'colsample_bytree': 0.9467721155563797, 'subsample': 0.7123051873743906, 'max_bin': 477, 'min_child_weight': 2.814005019556768}. Best is trial 53 with value: 47.77141155114695.
[I 2021-07-20 21:10:37,566] Trial 82 finished with value: 53.34104348030411 and parameters: {'learning_rate': 0.043735099537743176, 'reg_alpha': 3.2818101651236067e-06, 'reg_lambda': 0.06104491151326458, 'max_depth': 8, 'colsample_bytree': 0.9089110115743667, 'subsample': 0.7635014285465036, 'max_bin': 464, 'min_child_weight': 2.666233092566192}. Best is trial 53 with value: 47.77141155114695.
[I 2021-07-20 21:10:38,050] Trial 83 finished with value: 52.6807077992864 and parameters: {'learning_rate': 0.05573195893528013, 'reg_alpha': 2.002826318289135e-06, 'reg_lambda': 0.07617564039473361, 'max_de

...

[I 2021-07-20 21:19:44,830] Trial 921 finished with value: 58.23393002678366 and parameters: {'learning_rate': 0.08846959244730161, 'reg_alpha': 2.8115677200560437e-05, 'reg_lambda': 0.07228772863860317, 'max_depth': 3, 'colsample_bytree': 0.9385048644047738, 'subsample': 0.5825599096969354, 'max_bin': 284, 'min_child_weight': 1.4584324757658238}. Best is trial 909 with value: 44.92346862985306.
[I 2021-07-20 21:19:45,920] Trial 922 finished with value: 46.07173177574863 and parameters: {'learning_rate': 0.015707544709250933, 'reg_alpha': 1.408964127652412e-05, 'reg_lambda': 0.07757600465496552, 'max_depth': 6, 'colsample_bytree': 0.8470828775783751, 'subsample': 0.5732471305557437, 'max_bin': 342, 'min_child_weight': 1.3535151189581311}. Best is trial 909 with value: 44.92346862985306.
[I 2021-07-20 21:19:48,744] Trial 923 finished with value: 52.7616993238946 and parameters: {'learning_rate': 0.004370748242774076, 'reg_alpha': 1.346233272778615e-05, 'reg_lambda': 0.07825145353506271,

[I 2021-07-20 21:20:05,086] Trial 941 finished with value: 54.00500796021534 and parameters: {'learning_rate': 0.016851553959501907, 'reg_alpha': 8.773583101082458e-06, 'reg_lambda': 0.07730872935387738, 'max_depth': 7, 'colsample_bytree': 0.6606640909350652, 'subsample': 0.5613208978498532, 'max_bin': 355, 'min_child_weight': 1.4904604708246936}. Best is trial 909 with value: 44.92346862985306.
[I 2021-07-20 21:20:05,430] Trial 942 finished with value: 59.17181349201363 and parameters: {'learning_rate': 0.061952592434140044, 'reg_alpha': 1.5191708285617371e-05, 'reg_lambda': 0.07346425962790337, 'max_depth': 5, 'colsample_bytree': 0.8743766103947023, 'subsample': 0.5959915177689015, 'max_bin': 284, 'min_child_weight': 1.182884891048568}. Best is trial 909 with value: 44.92346862985306.
[I 2021-07-20 21:20:05,845] Trial 943 finished with value: 59.060755625492384 and parameters: {'learning_rate': 0.06487430955676261, 'reg_alpha': 2.9456194131061423e-05, 'reg_lambda': 0.0830220161870133

[I 2021-07-20 21:20:23,804] Trial 961 finished with value: 53.85266456283441 and parameters: {'learning_rate': 0.02106673283723998, 'reg_alpha': 2.4016170397357125e-05, 'reg_lambda': 0.08075216859224124, 'max_depth': 6, 'colsample_bytree': 0.5658147733136825, 'subsample': 0.5775569575439563, 'max_bin': 358, 'min_child_weight': 1.3188558411416773}. Best is trial 909 with value: 44.92346862985306.
[I 2021-07-20 21:20:25,250] Trial 962 finished with value: 56.14166370359789 and parameters: {'learning_rate': 0.015472587028454377, 'reg_alpha': 2.347550612874535e-05, 'reg_lambda': 0.07452102065532426, 'max_depth': 12, 'colsample_bytree': 0.62413846388308, 'subsample': 0.5585038984898856, 'max_bin': 365, 'min_child_weight': 1.1304965290180786}. Best is trial 909 with value: 44.92346862985306.
[I 2021-07-20 21:20:25,649] Trial 963 finished with value: 55.23672243326652 and parameters: {'learning_rate': 0.0810476962165697, 'reg_alpha': 2.282678078163308e-05, 'reg_lambda': 0.07147933773752597, '

[I 2021-07-20 21:20:43,020] Trial 981 finished with value: 60.40589181916052 and parameters: {'learning_rate': 0.015148265128489786, 'reg_alpha': 2.2512163618851748e-05, 'reg_lambda': 0.07083237544126791, 'max_depth': 8, 'colsample_bytree': 0.5222237793313329, 'subsample': 0.5543276803865871, 'max_bin': 294, 'min_child_weight': 1.4070409475913315}. Best is trial 909 with value: 44.92346862985306.
[I 2021-07-20 21:20:43,736] Trial 982 finished with value: 55.43079020396 and parameters: {'learning_rate': 0.02420320681952219, 'reg_alpha': 2.3961423903961798e-05, 'reg_lambda': 0.07424425298187863, 'max_depth': 7, 'colsample_bytree': 0.5654821533780372, 'subsample': 0.5740103705235249, 'max_bin': 282, 'min_child_weight': 1.300986311226987}. Best is trial 909 with value: 44.92346862985306.
[I 2021-07-20 21:20:44,573] Trial 983 finished with value: 52.685451058780444 and parameters: {'learning_rate': 0.021368103500677117, 'reg_alpha': 2.314018216821903e-05, 'reg_lambda': 0.07913396381687532, 

Best Score: 44.92346862985306
Best trial: {'learning_rate': 0.0787078150815904, 'reg_alpha': 9.315553344024968e-06, 'reg_lambda': 0.07202327567111269, 'max_depth': 7, 'colsample_bytree': 0.9464860467275263, 'subsample': 0.5607164000630228, 'max_bin': 286, 'min_child_weight': 1.5100289350554967}


In [46]:
X_train, X_valid, y_train, y_valid = train_test_split(dinner_train, y_dinner, test_size=0.2)

dinner_model = XGBRegressor(**dinner_param)
dinner_model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=False,
        )

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.9464860467275263, gamma=0,
       gpu_id=-1, importance_type='gain', interaction_constraints='',
       learning_rate=0.0787078150815904, max_bin=286, max_delta_step=0,
       max_depth=7, min_child_weight=1.5100289350554967, missing=nan,
       monotone_constraints='()', n_estimators=100, n_jobs=8,
       num_parallel_tree=1, objective='reg:squarederror', random_state=0,
       reg_alpha=9.315553344024968e-06, reg_lambda=0.07202327567111269,
       scale_pos_weight=1, subsample=0.5607164000630228,
       tree_method='exact', validate_parameters=1, verbosity=None)

In [51]:
dinner_predict = dinner_model.predict(dinner_test)
dinner_predict

array([314.35977, 390.58533, 219.94618, 508.57797, 460.73077, 498.00082,
       439.91437, 375.34412, 639.3592 , 561.4694 , 242.44696, 658.106  ,
       625.3772 , 382.7433 , 539.0808 , 380.37814, 651.0865 , 632.4505 ,
       317.88605, 543.0431 , 266.01813, 634.73016, 424.84207, 522.8469 ,
       382.13806, 645.03394, 639.05884, 503.4852 , 554.64294, 383.36978,
       693.1065 , 631.41003, 444.70734, 534.85175, 354.8829 , 687.5065 ,
       606.1427 , 309.107  , 463.46143, 220.54839, 679.2747 , 609.805  ,
       232.2445 , 456.11093, 316.9425 , 647.3791 , 640.8125 , 467.95502,
       480.22638, 339.94965], dtype=float32)

In [52]:
submission = pd.read_csv('../data/sample_submission.csv')

submission.iloc[:,1] = lunch_predict
submission.iloc[:,2] = dinner_predict
submission.head()

,일자,중식계,석식계
0,2021-01-27,912.140625,314.359772
1,2021-01-28,899.830078,390.585327
2,2021-01-29,675.871826,219.946182
3,2021-02-01,1162.267822,508.577972
4,2021-02-02,995.791382,460.730774


In [53]:
def compare_ans(DIR):
    answer = pd.read_csv(DIR)

    lunch_answer = np.array(answer.iloc[:,1])
    dinner_answer = np.array(answer.iloc[:,2])
    
    lunch_MAE = abs(submission.iloc[:,1] - lunch_answer).mean()
    dinner_MAE = abs(submission.iloc[:,2] - dinner_answer).mean()
    
    print("lunch_MAE : ", lunch_MAE)
    print("dinner_MAE : ", dinner_MAE)
    print("total_MAE : ", (lunch_MAE+dinner_MAE)/2)

In [2]:
import glob

files = glob.glob('../submission/*')

for file in files[:-1]:
    print(file, compare_ans(file), end="\n\n" )

NameError: name 'compare_ans' is not defined

In [1]:
print(files)

NameError: name 'files' is not defined

In [ ]:
import math

math.cos(math.pi*68.44/180)*69.5

# 저장

In [ ]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_optuna_AutoML.csv', index =False)